In [1]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_val, y_val = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_val, X_test = X_train/255., X_val/255. , X_test/255.

In [2]:
import keras_tuner as kt

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation='relu'))
        
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(
            loss = tf.keras.losses.sparse_categorical_crossentropy,
            optimizer = optimizer,
            metrics = ["accuracy"]
        )
    return model

In [3]:
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, overwrite=True,
    directory="my_fashion_mnist", project_name="my_random_search", seed=42
)
random_search_tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Trial 5 Complete [00h 00m 55s]
val_accuracy: 0.743399977684021

Best val_accuracy So Far: 0.8614000082015991
Total elapsed time: 00h 04m 45s


In [4]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [5]:
top3_parameters = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_parameters[0].values # best hyperparameters

{'n_hidden': 8,
 'n_neurons': 37,
 'learning_rate': 0.008547485565344062,
 'optimizer': 'sgd'}

In [6]:
best_trail = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trail.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8614000082015991


In [12]:
X_train_full = X_train_full/255.
history = best_model.fit(X_train_full, y_train_full, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: nan - accuracy: 0.1000
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.1000
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.1000
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.1000
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.1000
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 8/10
1875/1875 [==============================] - 5s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/step - loss: nan - accuracy: 0.1000


In [8]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (32, 784)                 0         
                                                                 
 dense (Dense)               (32, 37)                  29045     
                                                                 
 dense_1 (Dense)             (32, 37)                  1406      
                                                                 
 dense_2 (Dense)             (32, 37)                  1406      
                                                                 
 dense_3 (Dense)             (32, 37)                  1406      
                                                                 
 dense_4 (Dense)             (32, 37)                  1406      
                                                                 
 dense_5 (Dense)             (32, 37)                  1

In [10]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
test_accuracy

313/313 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.1000


0.10000000149011612

### Fine-Tune Data Preprocessing Hyperparameters Using Subclassing

In [19]:
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)
    
    def fit(self, hp, model, X, y, **kwargs):
        if hp.Boolean("normalize"):
            norm_layer = tf.keras.layers.Normalization()
            X = norm_layer(X)
        return model.fit(X, y, **kwargs)

In [21]:
hyperband_tuner = kt.Hyperband(
    MyClassificationHyperModel(), objective="val_accuracy", seed=42, max_epochs=10, factor=3, hyperband_iterations=2,
    overwrite=True, directory="my_fashion_mnist", project_name="hyperband"
)

In [22]:
from pathlib import Path

root_logdir = Path(hyperband_tuner.project_dir)/"tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(X_train, y_train, epochs=10, 
                       validation_data=(X_val, y_val),
                       callbacks=[early_stopping_cb, tensorboard_cb])

Trial 60 Complete [00h 01m 36s]
val_accuracy: 0.8428000211715698

Best val_accuracy So Far: 0.8870000243186951
Total elapsed time: 00h 38m 18s
